In [9]:
import json
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

# Load dataset from JSON file and prepare data for training and testing
def load_dataset(file_path):
    print(f"Loading dataset from file: {file_path}")
    with open(file_path, 'r') as file:
        data = json.load(file)

    captured_data = data['capturedData']

    # Extract accelerometer and gyroscope data
    accelerometerYAxis = [entry['accelerometerYAxis'] for entry in captured_data]

    # Create DataFrame with accelerometer data
    df = pd.DataFrame({'accelerometerY_FrontBack': accelerometerYAxis})
    print("Dataset loaded successfully.")
    return df

def labelling(data):
    print("Starting acceleration classification...")
    # Normalize data using Min-Max scaling
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data)

    # Apply K-means clustering to determine thresholds
    print("Applying K-means clustering...")
    kmeans = KMeans(n_clusters=2, random_state=42)
    kmeans.fit(scaled_data)
    thresholds = np.sort(kmeans.cluster_centers_.flatten())
    print("Thresholds determined by K-means clustering:")
    print(thresholds)
    
    # Apply classification function to scaled accelerometer data
    print("Classifying acceleration data...")
    data['classification'] = data['accelerometerY_FrontBack'].apply(lambda x: classify(x, thresholds))

    print("Acceleration classification completed.")
    return data

def classify(acceleration, thresholds):
    if acceleration > thresholds[1]:
        return 'aggressive'
    elif acceleration > thresholds[0]:
        return 'normal'
    else:
        return 'slow'

def main():
    datasets = [
        # List of dataset file paths
        r'..\Datasets\Acceleration\Till reached distance\10m\aggressive\BMW e36\acceleration 0m 10m aggressive-23-04-2023-19-42-30.json',
        r'..\Datasets\Acceleration\Till reached distance\10m\normal\BMW e36\acceleration 0m 10m normal-23-04-2023-19-30-17.json',
        r'..\Datasets\Acceleration\Till reached distance\10m\slow\BMW e36\acceleration 0m 10m slow-23-04-2023-19-19-20.json'
    ]

    for dataset in datasets:
        print("Load and train model for:", dataset)

        # Load dataset and preprocess
        data = load_dataset(dataset)

        # Classify acceleration behavior
        labelled_data = labelling(data)

        # Display classification result
        print("Classification result:")
        print(labelled_data)  # Display the first few rows of the classified data

if __name__ == "__main__":
    main()


Load and train model for: ..\Datasets\Acceleration\Till reached distance\10m\aggressive\BMW e36\acceleration 0m 10m aggressive-23-04-2023-19-42-30.json
Loading dataset from file: ..\Datasets\Acceleration\Till reached distance\10m\aggressive\BMW e36\acceleration 0m 10m aggressive-23-04-2023-19-42-30.json
Dataset loaded successfully.
Starting acceleration classification...
Applying K-means clustering...
Thresholds determined by K-means clustering:
[0.42189322 0.89404294]
Classifying acceleration data...
Acceleration classification completed.
Classification result:
    accelerometerY_FrontBack classification
0                   0.574472         normal
1                   0.175526           slow
2                   1.303585     aggressive
3                   0.753614         normal
4                   0.281722           slow
5                  -0.066736           slow
6                  -1.048308           slow
7                   1.421072     aggressive
8                  -0.586086       